In [1]:
try:
    import  pyyaml
except:
  !pip install pyyaml==5.1
from pycocotools.coco import COCO
import os
from PIL import Image
import skimage.io as io

import os
from pathlib import Path
import pandas as pd
import json
import time
import datetime

import numpy as np
import matplotlib.pyplot as plt
import os, json, cv2, random

# from google.colab.patches import cv2_imshow

import matplotlib.patches as patches
import matplotlib
import matplotlib.colors as mcolors
plt.rcParams.update({'font.size': 22})
style_label = 'fivethirtyeight' #plt.style.available 
try:
    import dsatools 
except:
    !pip install dsatools
finally:
    import dsatools.utilits as ut

In [9]:
def coco_datasets_info(list_images, list_anno):

    datasets_list = list()

    for i in range(len(list_anno)):

        if os.path.exists(list_anno[i]) and os.path.exists(list_images[i]): 


            dict_json = load_coco_json(json_file = list_anno[i], 
                                image_root = list_images[i], 
                                dataset_name = 'None',
                                extra_annotation_keys = None)
            dataset_info_dict = dict()      
            
            # dict_json = corret_coco_json_path(dict_json, list_images[i])

            name_dataset = os.path.split(os.path.split(os.path.split(list_anno[i])[0])[0])[1]

            dataset_info_dict['name']   = name_dataset
            dataset_info_dict['length'] = len(dict_json)

            df = pd.DataFrame(dict_json)

            dataset_info_dict['anno_path'] = list_anno[i]
            dataset_info_dict['image_path'] = list_images[i]

            dataset_info_dict['height'] = list(set(df['height']))
            dataset_info_dict['width']  = list(set(df['width']))

            coco=COCO(list_anno[i], )

            dataset_info_dict['anno_number'] = len(coco.anns)

            cats = coco.loadCats(coco.getCatIds())

            class_id = [cat['id'] for cat in cats]
            class_names = [cat['name'] for cat in cats]

            dataset_info_dict['class_id']  = class_id
            dataset_info_dict['class_names']  = class_names

            dataset_info_dict['COCO_obj']  = coco

            datasets_list +=[dataset_info_dict]
    return datasets_list

In [10]:
path = os.getcwd()
path = path[:path.find('src')]
path_2_project = os.path.join(path,'PitStonesData')
print(path_2_project)
list_project_dir =os.listdir(path_2_project)
list_project_dir

/home/kirill/Учеба/asbestos/PitStonesData


['pits_stones_020221',
 'pits_stones_300920',
 'pits_stones_161120',
 'pits_stones_111121',
 'asbestos_stone_lab_common_camera',
 'drive-download-20220131T131533Z-001.zip',
 'pits_stones_161220',
 'coco_eval',
 'stones_transport']

In [11]:
list_images = [os.path.join(path_2_project,data_pth,'images') for data_pth in list_project_dir]
list_anno   = [os.path.join(path_2_project,data_pth,'annotations','instances_default.json') for data_pth in list_project_dir]
list_images, list_anno

(['/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_020221/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_300920/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161120/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_111121/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/asbestos_stone_lab_common_camera/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/drive-download-20220131T131533Z-001.zip/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161220/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/coco_eval/images',
  '/home/kirill/Учеба/asbestos/PitStonesData/stones_transport/images'],
 ['/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_020221/annotations/instances_default.json',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_300920/annotations/instances_default.json',
  '/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161120/annotations/instances_default.json',
  '/home/kirill

In [14]:
import torch

TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION  = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION, '\n', 10*'-')

try:
    import detectron2
except:
  # Install detectron2 that matches the above pytorch version
  # See https://detectron2.readthedocs.io/tutorials/install.html for instructions
    !pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/$CUDA_VERSION/torch$TORCH_VERSION/index.html

  # If there is not yet a detectron2 release that matches the given torch + CUDA version, you need to install a different pytorch.
    exit(0)  # After installation, you may need to "restart runtime" in Colab. This line can also restart runtime
    
from detectron2.data.datasets    import register_coco_instances, load_coco_json

torch:  1.10 ; cuda:  cu102 
 ----------


In [16]:
coco_datasets_list = coco_datasets_info(list_images, list_anno);

loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.14s)
creating index...
index created!
loading annotations into memory...
Done (t=0.05s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.07s)
creating index...
index created!


In [21]:
df = pd.DataFrame(coco_datasets_list)
df

,name,length,anno_path,image_path,height,width,anno_number,class_id,class_names,COCO_obj
0,pits_stones_020221,13,/home/kirill/Учеба/asbestos/PitStonesData/pits...,/home/kirill/Учеба/asbestos/PitStonesData/pits...,[2048],[2592],685,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe2b02bf4f0>
1,pits_stones_300920,11,/home/kirill/Учеба/asbestos/PitStonesData/pits...,/home/kirill/Учеба/asbestos/PitStonesData/pits...,[2048],[2592],718,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe27134bbe0>
2,pits_stones_161120,68,/home/kirill/Учеба/asbestos/PitStonesData/pits...,/home/kirill/Учеба/asbestos/PitStonesData/pits...,[2048],[2592],4511,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe210c63940>
3,pits_stones_111121,19,/home/kirill/Учеба/asbestos/PitStonesData/pits...,/home/kirill/Учеба/asbestos/PitStonesData/pits...,[2048],[2592],2136,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe27134b100>
4,asbestos_stone_lab_common_camera,46,/home/kirill/Учеба/asbestos/PitStonesData/asbe...,/home/kirill/Учеба/asbestos/PitStonesData/asbe...,"[3456, 5184]","[5184, 3456]",1402,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe2b02afbe0>
5,pits_stones_161220,20,/home/kirill/Учеба/asbestos/PitStonesData/pits...,/home/kirill/Учеба/asbestos/PitStonesData/pits...,[2048],[2592],936,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe210c63d30>
6,stones_transport,134,/home/kirill/Учеба/asbestos/PitStonesData/ston...,/home/kirill/Учеба/asbestos/PitStonesData/ston...,[2048],[2592],2538,"[1, 2]","[stone, asbest]",<pycocotools.coco.COCO object at 0x7fe210c63d90>


In [23]:
print(df.length.sum())

311


In [52]:
dir(coco_datasets_list[0]['COCO_obj'])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'annToMask',
 'annToRLE',
 'anns',
 'catToImgs',
 'cats',
 'createIndex',
 'dataset',
 'download',
 'getAnnIds',
 'getCatIds',
 'getImgIds',
 'imgToAnns',
 'imgs',
 'info',
 'loadAnns',
 'loadCats',
 'loadImgs',
 'loadNumpyAnnotations',
 'loadRes',
 'showAnns']

In [90]:
i = 1
for coco_dataset in coco_datasets_list:
    for c in coco_dataset['COCO_obj'].dataset['images']:
#         print(i, c['file_name'])
        i+=1

In [108]:
coco_set['image_path']

'/home/kirill/Учеба/asbestos/PitStonesData/stones_transport/images'

In [107]:
for coco_set in coco_datasets_list:
    image_names = os.listdir(coco_set['image_path'])
    print(coco_set['image_path'])
#     for name in image_names:
#         print(name)

/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_020221/images
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_300920/images
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161120/images
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_111121/images
/home/kirill/Учеба/asbestos/PitStonesData/asbestos_stone_lab_common_camera/images
/home/kirill/Учеба/asbestos/PitStonesData/pits_stones_161220/images
/home/kirill/Учеба/asbestos/PitStonesData/stones_transport/images


In [103]:
image_names

['19_14_11_45_05-03-2020_1.jpg',
 '19_14_11_19_05-03-2020_1.jpg',
 '12_12_02_12_16-03-2020_1.jpg',
 '18_14_09_37_05-03-2020_1.jpg',
 '1_12_08_09_05-03-2020_1.jpg',
 '14_12_08_17_16-03-2020_1.jpg',
 '3_11_37_56_16-03-2020_1.jpg',
 '1_11_33_06_16-03-2020_1.jpg',
 '6_11_45_54_16-03-2020_1.jpg',
 '16_14_02_33_05-03-2020_1.jpg',
 '29_13_44_39_16-03-2020_1.jpg',
 '24_13_28_55_16-03-2020_1.jpg',
 '16_12_14_51_16-03-2020_1.jpg',
 '20_12_23_59_16-03-2020_1.jpg',
 '11_11_59_43_16-03-2020_1.jpg',
 '2_12_15_17_05-03-2020_1.jpg',
 '26_13_35_04_16-03-2020_1.jpg',
 '6_12_32_19_05-03-2020_1.jpg',
 '5_11_44_15_16-03-2020_1.jpg',
 '9_12_39_56_05-03-2020_1.jpg',
 '17_14_04_43_05-03-2020_1.jpg',
 '16_14_03_05_05-03-2020_1.jpg',
 '18_12_19_29_16-03-2020_1.jpg',
 '28_13_41_54_16-03-2020_1.jpg',
 '24_13_28_35_16-03-2020_1.jpg',
 '22_13_22_01_16-03-2020_1.jpg',
 '12_12_48_36_05-03-2020_1.jpg',
 '13_12_04_42_16-03-2020_1.jpg',
 '2_11_35_29_16-03-2020_1.jpg',
 '28_13_41_08_16-03-2020_1.jpg',
 '22_13_22_21_16-03